#Tarea 2
#Universidad Autónoma de Nuevo León
#Facultad de Ciencias Físico-Matemáticas
##Maestría en Ciencia de Datos
###Datos Masivos
####Miguel Alejandro Andres Bojórquez
####1837555


In [1]:

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!sudo apt update



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,451 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:1

In [4]:

import os
import sys
import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

import pandas as pd
from google.colab import drive
import os
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time

# **Explicación de base de datos**
El conjunto de datos que he elegido para mi tarea es un extenso registro de accidentes automovilísticos que abarca 49 estados de EE.UU. Este dataset se ha recopilado entre febrero de 2016 y marzo de 2023, utilizando múltiples APIs que proporcionan datos en tiempo real sobre incidentes de tráfico. Estas APIs obtienen información de diversas entidades, incluyendo departamentos de transporte, agencias de aplicación de la ley, cámaras de tráfico y sensores en las redes viales.

Este conjunto de datos es particularmente valioso ya que contiene aproximadamente 7.7 millones de registros de accidentes, lo que lo convierte en una base de datos muy grande para analizar. La riqueza en la cantidad de registros y la diversidad de columnas permiten realizar un análisis detallado sobre las tendencias y factores asociados a los accidentes.

He elegido este conjunto de datos porque es la base de datos más grande que encontré en los repositorios que revisé, y ofrece información tratada y relevante para llevar a cabo operaciones analíticas. Su amplitud tanto en registros como en columnas facilita el desarrollo de modelos predictivos, la identificación de patrones y la evaluación de políticas de seguridad vial. Esto lo convierte en un recurso ideal para realizar un análisis profundo y significativo sobre la accidentalidad en las carreteras.

https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

In [5]:

drive.mount('/content/drive')
spark = SparkSession.builder.appName("US_Accidentes").master("local[*]").getOrCreate()

file_path = "/content/drive/MyDrive/Datos Masivos/Datos/US_Accidents_March23.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)


Mounted at /content/drive


In [6]:
df.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [ ]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

In [ ]:
filtered_df = df.filter(df['State'] == 'OH').orderBy('Start_Time')
df_with_duration = filtered_df.withColumn(
    "Duration",
    (F.unix_timestamp(df['End_Time']) - F.unix_timestamp(df['Start_Time'])) / 60  # Diferencia en minutos
)

In [ ]:
# Convertir el DataFrame a RDD
rdd = df_with_duration.select("Severity", "Duration").rdd

In [ ]:
# Mapear a tuplas (Severity, Duration)
severity_duration_rdd = rdd.map(lambda x: (x[0], x[1]))

#Agrupar RDD tomando la severidad como llave y la duración como valores
rdd_agrupado = severity_duration_rdd.groupByKey()
#Imprimir el RDD agrupado, nos indica las llaves y que tiene un objeto pyspark dentro de cada llave
rdd_agrupado.collect()


[(1, <pyspark.resultiterable.ResultIterable at 0x78cf5c0be050>),
 (2, <pyspark.resultiterable.ResultIterable at 0x78cf5c0bfcd0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x78cf5c564820>),
 (4, <pyspark.resultiterable.ResultIterable at 0x78cf5c565b40>)]

In [ ]:
#Forma correcta de visualizar los datos en caso de desearlo
rdd_lista = rdd_agrupado.mapValues(list)
resultado = rdd_lista.collect()
resultado

[(1,
  [45.0,
   30.0,
   45.0,
   30.0,
   30.0,
   45.0,
   60.0,
   29.583333333333332,
   30.0,
   30.0,
   30.0,
   60.0,
   30.0,
   30.0,
   30.0,
   59.583333333333336,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   33.28333333333333,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   45.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   30.0,
   60.0,
   45.0,
   30.0,
   39.05,
   63.68333333333333,
   30.0,
   45.0,
   90.0,
   90.0,
   45.0,
   45.0,
   90.0,
   45.0,
   45.0,
   60.0,
   30.0,
   75.0,
   30.0,
   60.0,
   60.0,
   30.0,
   45.0,
   30.0,
   29.766666666666666,
   75.0,
   45.016666666666666,
   45.0,
   60.0,
   28.816666666666666,
   48.25,
   60.0,
   45.0,
   60.0,
   43.38333333333333,
   45.0,
   60.0,
   60.0,
   30.0,
   45.0,
   45.0,
   45.0,
   44.666666666666664,
   90.0,
   45.0,
   45.0,
   37.11666666666667,
   45.0,
   60.0,
   30.0,
  

In [ ]:

inicio = time.time() # Tiempo de procesamiento inicial
# Filtrar valores nulos en Duration
filtered_rdd = rdd.filter(lambda x: x[1] is not None)

# Calcular la suma y el conteo por Severidad. Está haciendo la suma del item 0 (duración) y la suma del item 1(dummy inicializada en 1 para contador)
severity_stats = filtered_rdd.map(lambda x: (x[0], (x[1], 1))).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Calcular la media por Severidad. Divido item 0 (suma de duración) sobre item 1 (contador)
mean_duration_by_severity = severity_stats.mapValues(lambda x: x[0] / x[1])

# Recoger resultados en results y los imprimo 1 por 1
results = mean_duration_by_severity.collect()
for severity, mean_duration in results:
    print(f"Severidad: {severity}, Duración Media: {mean_duration}")

fin = time.time() #Tiempo de procesamiento final
print("Tiempo con RDD:", fin - inicio)

Severidad: 3, Duración Media: 107.55083145357266
Severidad: 2, Duración Media: 123.41590276291477
Severidad: 4, Duración Media: 910.5527894110949
Severidad: 1, Duración Media: 39.42222547870266
Tiempo con RDD: 2.8090476989746094


In [ ]:
inicio = time.time() # Tiempo de procesamiento inicial
df_with_duration.groupBy("Severity").agg(F.mean("Duration").alias("Average_Duration")).orderBy('Severity').show()
fin = time.time() #Tiempo de procesamiento final
print("Tiempo con funciones regulares:", fin - inicio)

+--------+------------------+
|Severity|  Average_Duration|
+--------+------------------+
|       1| 39.42222547870266|
|       2|123.41590276291477|
|       3|107.55083145357266|
|       4| 910.5527894110949|
+--------+------------------+

Tiempo con funciones regulares: 109.06127429008484


Como podemos observar, hicimos el mismo procesamiento de datos para el mismo conjunto con funciones convencionales y RDDs, sin embargo, el tiempo de procesamiento fue de 2.8 segundos con RDDs comparado con los 109 segundos que se tomó con las funcionciones convencionales, disminuyendo el tiempo en 97%.

In [11]:
# Paso 1: Crear el RDD a partir del DataFrame
rdd = df.select("Severity", "State").rdd

# Paso 2: Contar accidentes por estado y severidad
accidents_count_rdd = rdd.map(lambda x: ((x['State'], x['Severity']), 1)) \
                          .reduceByKey(lambda a, b: a + b)

# Paso 3: Calcular el total de accidentes por estado
total_accidents_rdd = accidents_count_rdd.map(lambda x: (x[0][0], x[1])) \
                                          .reduceByKey(lambda a, b: a + b)

# Paso 4: Calcular la proporción de cada severidad
proportion_rdd = accidents_count_rdd.join(total_accidents_rdd) \
                                      .map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][0] / x[1][1]))

# Paso 5: Formatear el resultado final
final_table_rdd = proportion_rdd.map(lambda x: (x[0], x[1], x[2], f"{x[3]:.2%}"))

# Mostrar resultados
for state, severity, count, proportion in final_table_rdd.collect():
    print(f'State: {state}, Severity: {severity}, Count: {count}, Proportion: {proportion}')

In [15]:
total_accidents_rdd.collect()


[('VA', 303301),
 ('OK', 83647),
 ('SC', 382557),
 ('KY', 32254),
 ('ND', 3487),
 ('UT', 97079),
 ('AL', 101044),
 ('KS', 20992),
 ('NJ', 140719),
 ('DE', 14097),
 ('LA', 149701),
 ('WV', 13793),
 ('IL', 168958),
 ('CT', 71005),
 ('NY', 347960),
 ('AZ', 170609),
 ('TN', 167388),
 ('GA', 169234),
 ('IA', 26307),
 ('WY', 3757),
 ('TX', 582837),
 ('ME', 2698),
 ('NE', 28870),
 ('MI', 162191),
 ('SD', 289),
 ('NH', 10213),
 ('MO', 77323),
 ('PA', 296620),
 ('NM', 10325),
 ('OR', 179660),
 ('MN', 192084),
 ('RI', 16971),
 ('CA', 1741433),
 ('OH', 118115),
 ('ID', 11376),
 ('NC', 338199),
 ('WI', 34688),
 ('NV', 21665),
 ('DC', 18630),
 ('FL', 880192),
 ('MA', 61996),
 ('VT', 926),
 ('CO', 90885),
 ('AR', 22780),
 ('MT', 28496),
 ('MD', 140417),
 ('IN', 67224),
 ('MS', 15181),
 ('WA', 108221)]

In [18]:
# Paso 1: Contar accidentes por estado y severidad
accidents_count_rdd = rdd.map(lambda x: ((x['State'], x['Severity']), 1)) \
                          .reduceByKey(lambda a, b: a + b)

# Paso 2: Crear un RDD que contenga el conteo por estado y severidad
total_accidents_rdd = accidents_count_rdd.map(lambda x: (x[0][0], x[0][1], x[1]))

# Mostrar resultados
total_accidents_rdd.collect()

[('OH', 1, 1706),
 ('KY', 2, 18255),
 ('MD', 2, 103558),
 ('NC', 1, 5139),
 ('AZ', 2, 140553),
 ('IN', 1, 327),
 ('TN', 4, 3489),
 ('NC', 4, 10404),
 ('NH', 1, 82),
 ('OK', 4, 485),
 ('ND', 4, 17),
 ('IA', 4, 2069),
 ('MO', 2, 48766),
 ('DC', 2, 16489),
 ('PA', 1, 1731),
 ('PA', 4, 15867),
 ('NC', 3, 29590),
 ('NM', 3, 2849),
 ('WY', 2, 3028),
 ('ME', 4, 119),
 ('GA', 4, 12776),
 ('MO', 4, 2882),
 ('NJ', 2, 113315),
 ('NM', 1, 153),
 ('ME', 2, 2259),
 ('WY', 4, 547),
 ('VT', 4, 90),
 ('SC', 3, 42470),
 ('WA', 4, 4091),
 ('MN', 2, 160848),
 ('UT', 2, 79801),
 ('MS', 2, 10739),
 ('IN', 4, 5605),
 ('NH', 4, 254),
 ('SC', 1, 6175),
 ('DE', 3, 715),
 ('TX', 2, 450952),
 ('TN', 2, 133054),
 ('OR', 1, 1323),
 ('OH', 4, 6447),
 ('FL', 1, 7083),
 ('MO', 1, 270),
 ('DE', 1, 234),
 ('DC', 1, 143),
 ('DC', 4, 821),
 ('IA', 1, 58),
 ('MO', 3, 25405),
 ('MA', 1, 1500),
 ('DC', 3, 1177),
 ('NV', 1, 128),
 ('ID', 1, 1),
 ('ID', 4, 546),
 ('WY', 3, 182),
 ('NV', 4, 720),
 ('GA', 1, 1007),
 ('LA', 2, 12

Calculamos total de accidentes por estado y severidad, así como sus proporciones de severidad por estado. Con esto podemos saber que estado tiene mayor proporción de accidentes más o menos severos.

##Importancia de RDDs

En este ejercicio es de suma importancia utilizar RDDs para agilizar el cálculo, ya que si lo hubieramos hecho con funciones de pandas, podría tardar mucho tomando como base que calcular la duración promedio únicamente de Ohio tardó casi 2 minutos, de hacerse estos cálculos para todo el dataset podría tardar mucho más.

In [32]:
# Paso 1: Crear el RDD a partir del DataFrame
rdd = df.select("Severity", "State").rdd

# Paso 2: Contar accidentes por estado y severidad
accidents_count_rdd = rdd.map(lambda x: ((x['State'], x['Severity']), 1)) \
                          .reduceByKey(lambda a, b: a + b)

# Paso 3: Calcular el total de accidentes por estado
total_accidents_rdd = accidents_count_rdd.map(lambda x: (x[0][0], x[1])) \
                                          .reduceByKey(lambda a, b: a + b)

# Paso 4: Unir los conteos de severidad con los totales
# Cambiar la clave en accidents_count_rdd para que solo use el State
proportion_rdd = accidents_count_rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                      .join(total_accidents_rdd) \
                                      .map(lambda x: (x[0], x[1][0][0], x[1][0][1], x[1][1])) \
                                      .map(lambda x: (x[0], x[1], x[2], x[2] / x[3]))

# Paso 5: Formatear el resultado final
final_table_rdd = proportion_rdd.map(lambda x: (x[0], x[1], x[2], f"{x[3]:.2%}"))



In [33]:
# Mostrar resultados
for state, severity, count, proportion in final_table_rdd.collect():
    print(f'State: {state}, Severity: {severity}, Count: {count}, Proportion: {proportion}')

State: VA, Severity: 2, Count: 230660, Proportion: 76.05%
State: VA, Severity: 4, Count: 18202, Proportion: 6.00%
State: VA, Severity: 1, Count: 3115, Proportion: 1.03%
State: VA, Severity: 3, Count: 51324, Proportion: 16.92%
State: OK, Severity: 4, Count: 485, Proportion: 0.58%
State: OK, Severity: 1, Count: 1303, Proportion: 1.56%
State: OK, Severity: 2, Count: 75059, Proportion: 89.73%
State: OK, Severity: 3, Count: 6800, Proportion: 8.13%
State: KS, Severity: 2, Count: 14837, Proportion: 70.68%
State: KS, Severity: 1, Count: 34, Proportion: 0.16%
State: KS, Severity: 4, Count: 800, Proportion: 3.81%
State: KS, Severity: 3, Count: 5321, Proportion: 25.35%
State: NJ, Severity: 2, Count: 113315, Proportion: 80.53%
State: NJ, Severity: 3, Count: 21318, Proportion: 15.15%
State: NJ, Severity: 4, Count: 5887, Proportion: 4.18%
State: NJ, Severity: 1, Count: 199, Proportion: 0.14%
State: IL, Severity: 1, Count: 1620, Proportion: 0.96%
State: IL, Severity: 4, Count: 4317, Proportion: 2.56%

In [35]:
sorted_final_table_rdd = final_table_rdd.sortBy(lambda x: (x[0], x[1], x[2]))  # Ordenar en orden descendente de proporción

In [36]:
sorted_final_table_rdd.collect()

[('AL', 1, 681, '0.67%'),
 ('AL', 2, 76993, '76.20%'),
 ('AL', 3, 21542, '21.32%'),
 ('AL', 4, 1828, '1.81%'),
 ('AR', 1, 37, '0.16%'),
 ('AR', 2, 19239, '84.46%'),
 ('AR', 3, 677, '2.97%'),
 ('AR', 4, 2827, '12.41%'),
 ('AZ', 1, 7389, '4.33%'),
 ('AZ', 2, 140553, '82.38%'),
 ('AZ', 3, 17551, '10.29%'),
 ('AZ', 4, 5116, '3.00%'),
 ('CA', 1, 10284, '0.59%'),
 ('CA', 2, 1445833, '83.03%'),
 ('CA', 3, 271814, '15.61%'),
 ('CA', 4, 13502, '0.78%'),
 ('CO', 1, 970, '1.07%'),
 ('CO', 2, 56105, '61.73%'),
 ('CO', 3, 26306, '28.94%'),
 ('CO', 4, 7504, '8.26%'),
 ('CT', 1, 72, '0.10%'),
 ('CT', 2, 50392, '70.97%'),
 ('CT', 3, 16369, '23.05%'),
 ('CT', 4, 4172, '5.88%'),
 ('DC', 1, 143, '0.77%'),
 ('DC', 2, 16489, '88.51%'),
 ('DC', 3, 1177, '6.32%'),
 ('DC', 4, 821, '4.41%'),
 ('DE', 1, 234, '1.66%'),
 ('DE', 2, 11596, '82.26%'),
 ('DE', 3, 715, '5.07%'),
 ('DE', 4, 1552, '11.01%'),
 ('FL', 1, 7083, '0.80%'),
 ('FL', 2, 755895, '85.88%'),
 ('FL', 3, 104065, '11.82%'),
 ('FL', 4, 13149, '1.49%')